In [1]:
import pandas as pd
import numpy as np 
import openpyxl
import math
from pathlib import Path

In [2]:
#Import data
df = pd.read_excel(r'C:\Users\Acer\OneDrive - Institut Teknologi Bandung\Magang BI\Data\Web Scraping Hotel\data fix 2\data_fix_Bali2.xlsx', sheet_name="Hotel_2",engine = "openpyxl")
data = np.array(df)
#print(df)

In [3]:
#Latitude : 56
#Longitude : 57
#LD : 58
#PD H+1 : 59
#PD H+7 : 60
#avg H+1 : 61
#avg H+7 : 62
#avg rating : 63

n = 10

def jarak (la0,lo0,la1,lo1):
    distance = math.sqrt(((la0-la1)**2)+((lo0-lo1)**2))
    return distance

def harga (p0,p1):
    price = abs(p0-p1)
    return price

for i in range (2,len(data[:,56])):
    
    array_jarak = np.full([2,len(data[:,56])],np.nan)
    
    for j in range (2,len(data[:,56])):
        if i != j:
            array_jarak[0,j] = jarak(data[i,56],data[i,57],data[j,56],data[j,57])
            array_jarak[1,j] = j
    
    dummy_jarak = array_jarak[:,np.argsort(array_jarak[0])][0:n]
    nearest_jarak = dummy_jarak[:,0:n]
    LD = (np.sum(nearest_jarak[0,:]))/n
    data[i,58] = LD
    
    #Cari perbedaan harga di hotel-hotel terdekat  
    
    PD_H1 = np.full(len(data[2,59:69]),np.nan)  #Array PD H+1, panjangnya = 10
    PD_H7 = np.full(len(data[2,69:79]),np.nan) #Array PD H+7, panjangnya = 10  
    
    for k in range (10):
        m1 = 0
        sum_PD1 = 0
        m7 = 0
        sum_PD7 = 0          
        for j in range (len(nearest_jarak[1])):
            z = int(nearest_jarak[1,j])                    
            if data[i,k+25] > 0 and data[z,k+25] > 0 : #Buat H+1
                PD_temp1 = harga(data[i,k+25],data[z,k+25])
                m1 = m1+1
                sum_PD1 = sum_PD1 + PD_temp1            
            if data[i,k+35] > 0 and data[z,k+35] > 0 : #Buat H+7
                PD_temp7 = harga(data[i,k+35],data[z,k+35])
                m7 = m7+1
                sum_PD7 = sum_PD7 + PD_temp7   
        if m1 > 0 :
            PD_H1[k] = sum_PD1/m1
        if m7 > 0 :
            PD_H7[k] = sum_PD7/m7
            
    data[i,59:69] = PD_H1
    data[i,69:79] = PD_H7

In [4]:
df_data = pd.DataFrame(data)
df_data.to_excel(r"C:\Users\Acer\Downloads\data_ld_pd2.xlsx")